In [1]:
import tensorflow as tf
import datetime
import os
import numpy as np
import ctypes
from PIL import Image

In [ ]:
#### ------------------------------------------- ####
#### ------------- LOAD THE DATA --------------- ####
#### ------------------------------------------- ####

In [ ]:
MAIN_DATASET_FOLDER = "..\\dataset\\"

TEST_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "test")
TRAIN_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "train")

TEST_ESPAGNE_FOLDER = os.path.join(TEST_SUBFOLDER, "espagne")
TEST_FRANCE_FOLDER = os.path.join(TEST_SUBFOLDER, "france")
TEST_JAPON_FOLDER = os.path.join(TEST_SUBFOLDER, "japon")

TRAIN_ESPAGNE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "espagne")
TRAIN_FRANCE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "france")
TRAIN_JAPON_FOLDER = os.path.join(TRAIN_SUBFOLDER, "japon")


In [ ]:
def fill_x_and_y_with_images_and_labels(folder, classe, dataset):

    for file in os.listdir(folder):

        file_path = os.path.join(folder, file)
        image = Image.open(file_path)
        image = image.resize((64, 64))
        if image.mode in ("RGB", "P"): image = image.convert("RGBA")
        im_arr = np.array(image).flatten()
        im_arr = im_arr / 255.0

        completed_dataset = {}
        completed_dataset["value"] = im_arr
        completed_dataset["classe"] = classe

        dataset.append(completed_dataset.copy())

In [ ]:
def import_dataset(folder, classe):

    dataset = []

    fill_x_and_y_with_images_and_labels(folder, classe, dataset)

    return dataset

In [ ]:
x_test = []
y_test = []

x_train = []
y_train = []

class_espagne_expected_outputs = [1, 0, 0]
dataset_test_for_espagne = import_dataset(TEST_ESPAGNE_FOLDER, class_espagne_expected_outputs)
dataset_train_for_espagne = import_dataset(TRAIN_ESPAGNE_FOLDER, class_espagne_expected_outputs)

class_france_expected_outputs = [0, 1, 0]
dataset_test_for_france = import_dataset(TEST_FRANCE_FOLDER, class_france_expected_outputs)
dataset_train_for_france = import_dataset(TRAIN_FRANCE_FOLDER, class_france_expected_outputs)

class_japon_expected_outputs = [0, 0, 1]
dataset_test_for_japon = import_dataset(TEST_JAPON_FOLDER, class_japon_expected_outputs)
dataset_train_for_japon = import_dataset(TRAIN_JAPON_FOLDER, class_japon_expected_outputs)

final_dataset_test = dataset_test_for_espagne + dataset_test_for_france + dataset_test_for_japon
final_dataset_train = dataset_train_for_espagne + dataset_train_for_france + dataset_train_for_japon

In [ ]:
for value in final_dataset_test:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_test.append(the_image)
    y_test.append(the_classe)

for value in final_dataset_train:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_train.append(the_image)
    y_train.append(the_classe)

dataset_inputs = x_train
dataset_outputs = y_train

initiate_dataset_inputs_size = len(dataset_inputs[0])
initiate_dataset_inputs_type = ctypes.c_float * initiate_dataset_inputs_size

classes_initiate_dataset_inputs_size = initiate_dataset_inputs_size * 2

In [ ]:
inputs_result = []

for sublist in dataset_inputs:
    for item in sublist:
        inputs_result.append(item)

dataset_inputs = inputs_result

outputs_result = []

for sublist in dataset_outputs:
    for item in sublist:
        outputs_result.append(item)

dataset_outputs = outputs_result

arr = [initiate_dataset_inputs_size, classes_initiate_dataset_inputs_size, 3]
arr_size = len(arr)
arr_type = ctypes.c_int * arr_size

In [ ]:
# Convert train/test to array
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
#### ------------------------------------------- ####
#### ------------LOAD MODELS---------------------#### 
#### ------------------------------------------- ####

In [ ]:
model_pmc_a = tf.keras.models.load_model("../models/keras_models/model_a.h5")

In [ ]:
model_pmc_b = tf.keras.models.load_model("../models/keras_models/model_b.h5")

In [ ]:
model_pmc_c = tf.keras.models.load_model("../models/keras_models/model_c.h5")

In [ ]:
model_pmc_d = tf.keras.models.load_model("../models/keras_models/model_d.h5")

In [ ]:
#### ------------------------------------------- ####
#### ------ DEFINE THE TRAINING ALGORITHM ------ #### 
#### ------------------------------------------- ####

In [ ]:
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=1e-3)
LOSS = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
METRICS = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
]

In [ ]:
model_pmc_a.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_b.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_c.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_d.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

In [ ]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (A) ------------- #### 
#### ------------------------------------------- ####

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_a.h5', save_best_only=True)

In [ ]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_a_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_a = model_pmc_a.fit(
    x=x_train, 
    y=y_train, 
    epochs=1000, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
        
        
    ]

)

In [ ]:
history_a.history.keys()

In [ ]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (B) ------------- #### 
#### ------------------------------------------- ####

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_b.h5', save_best_only=True)

In [ ]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_b_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_b = model_pmc_b.fit(
    x=x_train, 
    y=y_train, 
    epochs=1000, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

In [ ]:
history_b.history.keys()

In [ ]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (C) ------------- #### 
#### ------------------------------------------- ####

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_c.h5', save_best_only=True)

In [ ]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_c_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_c = model_pmc_c.fit(
    x=x_train, 
    y=y_train, 
    epochs=1000, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

In [ ]:
history_c.history.keys()

In [ ]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (D) ------------- #### 
#### ------------------------------------------- ####

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_d.h5', save_best_only=True)

In [ ]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_d_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_d = model_pmc_d.fit(
    x=x_train, 
    y=y_train, 
    epochs=1000, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

In [ ]:
history_d.history.keys()